<h1><b><center>DESAFIO DE INFERÊNCIA - TERA</center></b></h1>

#### OBSERVAÇÃO:

Correção dos códigos de raça/etnia do dicionário de demo_phq:
- 1 - Mexicano-americano
- 2 - Negro não-hispânico
- 3 - Branco não-hispânico
- 4 - Outro
- 5 - Outro-Hispânico

## Importando as bibliotecas e carregando as bases de dados

In [ ]:
import pandas as pd
import numpy as np
import statistics
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
demo_phq = pd.read_csv('DEMO_PHQ.csv')
pag_hei = pd.read_csv('PAG_HEI.csv')

In [ ]:
demo_phq.head()

In [ ]:
demo_phq.info()

In [ ]:
pag_hei.head()

In [ ]:
pag_hei.info()

## Colocando a variável SEQN como index e unindo as DB 

In [ ]:
df_merged = pd.merge(demo_phq, pag_hei, on = 'SEQN')
df_merged

In [ ]:
# Avaliando o percentual de casos faltantes

df_merged.isnull().mean()*100

In [ ]:
df_merged.drop(columns = ['SEQN']).describe(percentiles = [.01, .05, .25, .5, .75, .95, .99]).round(2)

## Avaliando as variáveis

In [ ]:
# Distribuição de raça/etnia:

df_merged[['RIDRETH1']].value_counts(sort = False)

In [ ]:
# Distribuição de escolaridade

df_merged[['DMDEDUC']].value_counts(sort = False)

In [ ]:
# Distribuição de renda

df_merged[['INDFMINC']].value_counts(sort = False)

In [ ]:
# Criando uma coluna sem 77 (se recusou a responder) e 99 (não sabe)

df_merged['RENDA_MAIOR_MENOR_20000'] = df_merged['INDFMINC'].loc[(df_merged['INDFMINC'] != 77) & 
                                                           (df_merged['INDFMINC'] != 99)]

df_merged[['RENDA_MAIOR_MENOR_20000']].value_counts(sort = False)

In [ ]:
# Agrupando as rendas maiores e menores que 20000 em novas variáveis

menor_20000 = []
maior_20000 = []

for _, renda in enumerate(df_merged['RENDA_MAIOR_MENOR_20000']):
    if renda <= 4 or renda == 13 and renda != 12:
        menor_20000.append(renda)
    elif renda != 13:
        maior_20000.append(renda)

menor_20000 = pd.Series(menor_20000)
maior_20000 = pd.Series(maior_20000)

In [ ]:
# Distribuição de grupo de aderência

df_merged[['ADHERENCE']].value_counts(sort = False)

Agrupando/recategorizando variáveis qualitativas:

1) Substituindo as categorias 7 e 9 por missing para as variáveis DPQ0X0 e DMDEDUC

2) Agrupando as categorias 2 e 5 na variável RIDRETH1


In [ ]:
replace_map = {
  "DPQ010": {7: np.nan, 9: np.nan},
  "DPQ020": {7: np.nan, 9: np.nan},
  "DPQ030": {7: np.nan, 9: np.nan},
  "DPQ040": {7: np.nan, 9: np.nan},
  "DPQ050": {7: np.nan, 9: np.nan},
  "DPQ060": {7: np.nan, 9: np.nan},
  "DPQ070": {7: np.nan, 9: np.nan},
  "DPQ080": {7: np.nan, 9: np.nan},
  "DPQ090": {7: np.nan, 9: np.nan},
  "RIDRETH1": {5: 2},
  "DMDEDUC": {7: np.nan, 9: np.nan}
}

df_merged_2 = df_merged.replace(replace_map)

Como o percentual de dados nulos é abaixo de 10% para todas as variáveis, a análise será feita com os dados completos

In [ ]:
# Criando a variável PHQ9 que ontém a soma das variáveis das colunas DPQ010 a DPQ090

df_merged_2['PHQ9'] = df_merged_2.iloc[:, 1:10].sum(axis = 1, skipna = False)
phq9 = df_merged['PHQ9']

In [ ]:
conditions = [
  (df_merged_2['PHQ9'].isna()),
  (df_merged_2['PHQ9'] <= 5),
  (df_merged_2['PHQ9'] > 5) & (df_merged_2['PHQ9'] <= 9),
  (df_merged_2['PHQ9'] > 9) & (df_merged_2['PHQ9'] <= 14),
  (df_merged_2['PHQ9'] > 14) & (df_merged_2['PHQ9'] <= 19),
  (df_merged_2['PHQ9'] > 19)]
values = [np.nan, 0, 1, 2, 3, 4]

# Criando a variável phq_grp que representar os grupos com depressão
df_merged_2['phq_grp'] = np.select(conditions, values)

In [ ]:
# Como 3 e 4 tem baixas incidencias, será feita uma substituição onde
# 0 - Sem sintomas
# 1 - Sintomas leves
# 2 - Sintomas moderados a severos

df_merged_2['DEPRESINTGROUP'] = df_merged_2['phq_grp'].replace([3, 4], 2)
df_merged_2[['DEPRESINTGROUP']].value_counts(sort = False)

In [ ]:
# Distribuição dos escores para PHQ9

df_merged_2.iloc[:, 1:10].agg(['value_counts'])

## Análise Univariada

In [ ]:
# Respostas das perguntas

fig, axes = plt.subplots(3, 3, figsize = (15,15))
colunas = df_merged.columns[1:10].tolist()
matrix = np.array([[1, 2, 3],[4, 5, 6], [7, 8, 9]])
for col in colunas:
    idx = colunas.index(col) + 1
    linha = np.where(matrix == idx)[0][0]
    coluna = np.where(matrix == idx)[1][0]
    sns.histplot(df_merged_2[col], ax = axes[linha, coluna],stat = 'percent', discrete = True)

Através dos gráficos acima, é possível observar que todos possuem características semelhantes, com a resposta "0 = Nenhuma vez" em maior evidência. Porém, nos gráficos das perguntas 3 e 4, é possível observar a coluna com resposta "1 = Menos de uma semana" com um maior aumento em comparação aos outros gráficos.

A pergunta 3 trata-se de "dificuldade para pegar no sono ou permanecer dormindo ou dormiu mais do que o costume" e pergunta 4 sobre "se sentiu cansado(a) ou com pouca energia". Tal aumento pode ser devido a uma rotina atarefada e estressante, não sendo necessariamente relacionado a depressão.

In [ ]:
# Gênero

plt.figure(figsize = (10, 5))
plt.xticks([1, 2], ['Masculino', 'Feminino'])
sns.histplot(df_merged_2['RIAGENDR'], stat = 'percent', discrete = True);

As pessoas que mais participaram da pesquisa foram do gênero feminino, porém, a diferença entre a quantidade de pessoas de gêneros diferentes não é tão significativa.

Através da aplicação de contagem de elementos únicos da biblioteca numpy, é possível observar a quantidade exata de pessoas do gênero feminino e masculino, havendo 2773 pessoas do gênero feminino e 2561 pessoas do gênero masculino, uma diferença de 212 pessoas a mais do gênero feminino.

In [ ]:
# Idade

plt.figure(figsize = (10, 5))
sns.displot(df_merged_2, x = 'RIDAGEYR', kde = True);

O boxplot das idades das pessoas que participaram da pesquisa retorna que, em média, os entrevistados possuiam aproximadamente 45 anos, com um desvio padrão de aproximadamente 20,14 anos. Já a mediana é de 43 e a maioria dos entrevistados possuem 18 anos, o que infere em uma distribuição significativa de idade entre os entrevistados.

In [ ]:
# Raça/etnia

plt.figure(figsize = (10, 5))
sns.histplot(df_merged_2['RIDRETH1'], stat = 'percent', discrete = True);

Através do gráfico acima, é possível observar que a maior parte dos entrevistados é de brancos não-hispânicos, seguido por outras raças/etnias e mexicanos-americanos não-hispânicos. As pessoas negras não-hispânicas e de outras etnias hispânicas estão em menor proporção.

In [ ]:
# Escolaridade

plt.figure(figsize = (10, 5))
escolaridade_sem_7_9 = df_merged_2['DMDEDUC'].loc[(df_merged_2['DMDEDUC'] != 7) & (df_merged_2['DMDEDUC'] != 9)]
sns.histplot(escolaridade_sem_7_9, stat = 'percent', discrete = True);

A maior parte dos entrevistados possui ensino superior incompleto, seguido pelos que tem, respectivamente, ensino médio, superior completo ou maior, 9 ao 12º ano e por último, menor que 9º ano.

In [ ]:
# Renda anual familiar - maior que 20000

plt.figure(figsize = (10, 5))
sns.histplot(maior_20000, stat = 'percent', discrete = True);

In [ ]:
# Renda anual familiar - menor que 20000

plt.figure(figsize = (10, 5))
sns.histplot(menor_20000, stat = 'percent', discrete = True);

Das pessoas com renda superior a 20000, a maior parte dos entrevistados possui renda superior a 75000 e das com renda inferior a 20000, a maioria possui renda entre 10000 e 14999.

In [ ]:
# Tempo total semanal de atividade aeróbica moderada-vigorosa em minutos segundo o PAG

'''Como uma semana tem no máximo 168 horas (10080 minutos), 
o valor máximo de PAG_MINW não parece ser um valor possível.
Nesse caso, será feito o truncamento do valor de PAG_MINW em 3600 minutos (60h semanais)
e logo em seguida transformado em horas.'''

df_merged_2['PAG_MINW_trunc'] = np.where(df_merged_2['PAG_MINW'] > 3600, 3600, df_merged_2['PAG_MINW'])
df_merged_2["PAG_HRW"] = df_merged_2["PAG_MINW_trunc"]/60

plt.figure(figsize = (10, 5))
sns.displot(df_merged_2, x = 'PAG_HRW', kde = True);

In [ ]:
# Aderência

plt.figure(figsize = (10, 5))
sns.histplot(df_merged_2['ADHERENCE'], stat = 'percent', discrete = True);

O gráfico da aderência dá uma ideia do porquê haver tantos outliers no gráfico de tempo total semanal de atividade aeróbica já que o tempo adequado de exercício por semana é bem menor se comparado ao tempo abaixo de 150 min/semana ou acima de 300 min/semana.

In [ ]:
# Plotando os histogramas para as variáveis relativas ao HEI com medidas de 0 a 5

colunas = df_merged_2.columns[17:21].tolist()
colunas.extend(df_merged_2.columns[23:25].tolist())

fig, axes = plt.subplots(2,3, figsize=(15,15))
matriz = np.array([[1, 2, 3],[4, 5, 6]])
for col in colunas:
    idx = colunas.index(col)+1
    linha = np.where(matriz==idx)[0][0]
    coluna = np.where(matriz==idx)[1][0]
    sns.histplot(df_merged_2[col], ax=axes[linha,coluna], stat = 'percent', discrete = True)

- Em geral, os gráficos encontram-se com uma boa quantidade de pessoas que atingiram a quantidade máxima ingerida destes tipos de alimento no momento de realização da pesquisa.
- Observa-se nos gráficos de 'verduras verde-escuras e leguminosas', 'frutas totais', 'frutas in natura' e 'plantas marinhas e proteínas vegetais' uma grande quantidade de pessoas que não ingeriram estes tipos de alimentos, sendo mais predominante nos gráficos de 'verduras verde-escuras e leguminosas' e 'frutas in natura'
- Houve uma grande quantidade de pessoas que atingiram o máximo de proteína, havendo poucas pessoas que não ingeriram nenhuma porção ou ingeriram menos deste tipo de alimento enquanto era realizada a pesquisa. Isto pode ser em razão que a maioria dos alimentos proteicos ingeridos é de origem animal e é mais "fácil" ingerir este tipo de alimento que vegetais e frutas.

In [ ]:
# Plotando os histogramas para as variáveis relativas ao HEI com medidas de 0 a 10

colunas = df_merged_2.columns[21:23].tolist()
colunas.extend(df_merged_2.columns[25:30].tolist())

fig, axes = plt.subplots(3,3, figsize = (15,15))
matriz = np.array([[1, 2, 3],[4, 5, 6], [7, 8, 9]])
for col in colunas:
    idx = colunas.index(col)+1
    linha = np.where(matriz==idx)[0][0]
    coluna = np.where(matriz==idx)[1][0]
    sns.histplot(df_merged_2[col], ax=axes[linha,coluna], stat = 'percent', discrete = True)

Nestes gráficos, observa-se que há uma grande quantidade de pessoas que não ingeriram grãos integrais, enquanto que alimentos prejudiciais a saúde foram ingeridas em maior quantidade.

In [ ]:
# Escore total

plt.figure(figsize = (10, 5))
sns.boxplot(df_merged_2['HEI2015_TOTAL_SCORE']);

In [ ]:
# PHQ9

plt.figure(figsize = (10, 5))
sns.histplot(df_merged_2['PHQ9'], stat = 'percent', discrete = True);

In [ ]:
# Grupo com sintomas depressivos

plt.figure(figsize = (10, 5))
sns.histplot(df_merged_2['DEPRESINTGROUP'], stat = 'percent', discrete = True);

Com os dados obtidos, a maioria dos entrevistados não possuem sintomas depressivos.

## Análise Bivariada e Teste de Hipóteses

In [ ]:
# Importando as bibliotecas necessárias

from scipy.stats import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [ ]:
# Separando variáveis qualitativas e quantitativas

var_quant = [
    "RIDAGEYR", 
    "INDFMINC", 
    "PAG_MINW", 
    "HEI2015C1_TOTALVEG",
    "HEI2015C2_GREEN_AND_BEAN",
    "HEI2015C3_TOTALFRUIT",
    "HEI2015C4_WHOLEFRUIT",
    "HEI2015C5_WHOLEGRAIN",
    "HEI2015C6_TOTALDAIRY",
    "HEI2015C7_TOTPROT",
    "HEI2015C8_SEAPLANT_PROT",
    "HEI2015C9_FATTYACID",
    "HEI2015C10_SODIUM",
    "HEI2015C11_REFINEDGRAIN",
    "HEI2015C12_SFAT",
    "HEI2015C13_ADDSUG",
    "HEI2015_TOTAL_SCORE",
    "PHQ9"]

var_quali = [
    "RIAGENDR",
    "RIDRETH1",
    "DMDEDUC",
    "ADHERENCE",
    "DEPRESINTGROUP"
]

label_quali = {
  "RIAGENDR": {1: 'Masculino', 2: 'Feminino'},
  "RIDRETH1": {1: 'Americano Mexicano', 2: 'Outro', 3: 'Branco \n não hispânico', 4: 'Negro \n não hispânico'},
  "DMDEDUC": {1: "< 9 ano", 2: "9-12 ano", 3: "Ensino \n médio", 4: "Superior \n incompleto", 5: "Superior \n completo"},
  "ADHERENCE": {1: 'Baixo', 2: 'Adequado', 3: 'Acima'},
  "DEPRESINTGROUP": {0: "Sem sintomas", 1: "Sintomas \n leves", 2: "Sintomas \n moderados-severos"}
}

### Perfil com maiores incidências de sintomas depressivos

In [ ]:
# Função para construir boxplot

def grafico_boxplot_grp(data, variable, label):
    if label == '': label = variable
    sns.boxplot(x = 'DEPRESINTGROUP', y = variable, data = data)
    plt.ylabel(label)
    plt.show()

In [ ]:
# Idade e sintomas depressivos

grafico_boxplot_grp(df_merged_2.replace(label_quali), "RIDAGEYR", "Idade")

- H0: Não existe diferença entre os grupos de depressão com relação à média de idade
- H1: Existe pelo menos uma diferença na média de idade entre os grupos de sintomas de depressão

In [ ]:
df_teste = df_merged_2[["DEPRESINTGROUP", "RIDAGEYR"]].dropna()

stat, p = f_oneway(df_teste[(df_teste.DEPRESINTGROUP == 0)]["RIDAGEYR"],
                   df_teste[(df_teste.DEPRESINTGROUP == 1)]["RIDAGEYR"],
                   df_teste[(df_teste.DEPRESINTGROUP == 2)]["RIDAGEYR"])

if p < 0.05:
    print('H0 rejeitada: Existe pelo menos uma diferença na média de idade entre os grupos de sintomas de depressão')
else:
    print('H0 aceita: Não existe diferença entre os grupos de depressão com relação à média de idade')

- H0: Não existe diferença entre os grupos de depressão com relação à média da renda
- H1: Existe pelo menos uma diferença na média da renda entre os grupos de sintomas de depressão

In [ ]:
# Renda e sintomas depressivos

grafico_boxplot_grp(df_merged_2.replace(label_quali), "INDFMINC", "Renda Anual Familiar (US$)")

In [ ]:
df_teste = df_merged_2[["DEPRESINTGROUP", "INDFMINC"]].dropna()

stat, p = f_oneway(df_teste[(df_teste.DEPRESINTGROUP == 0)]["INDFMINC"],
                   df_teste[(df_teste.DEPRESINTGROUP == 1)]["INDFMINC"],
                   df_teste[(df_teste.DEPRESINTGROUP == 2)]["INDFMINC"])

if p < 0.05:
    print('H0 rejeitada: Existe pelo menos uma diferença na média da renda entre os grupos de sintomas de depressão')
else:
    print('H0 aceita: Não existe diferença entre os grupos de depressão com relação à média da renda')

In [ ]:
# Como existe pelo menos uma diferença, será realizado o teste de Tukey

tukey = pairwise_tukeyhsd(df_teste['INDFMINC'], df_teste['DEPRESINTGROUP'])
print(tukey)

### Hábitos saudáveis e sintomas depressivos

In [ ]:
# Como a variável PAG_HRW é bastante assimétrica, foi criada uma variável com o log a fim de normalizá-la

df_merged_2["PAG_HRW_log"] = np.log(df_merged_2["PAG_HRW"] + 1)
grafico_boxplot_grp(df_merged_2.replace(label_quali), "PAG_HRW_log", "Log Atividade Física (h/semana)")

- H0: Não existe diferença entre os grupos de depressão com relação à média das horas em atividade
- H1: Existe pelo menos uma diferença na média das horas em atividade entre os grupos de sintomas de depressão

In [ ]:
df_teste = df_merged_2[["DEPRESINTGROUP", "PAG_HRW_log"]].dropna()

stat, p = f_oneway(df_teste[(df_teste.DEPRESINTGROUP == 0)]["PAG_HRW_log"],
                   df_teste[(df_teste.DEPRESINTGROUP == 1)]["PAG_HRW_log"],
                   df_teste[(df_teste.DEPRESINTGROUP == 2)]["PAG_HRW_log"])

if p < 0.05:
    print('H0 rejeitada: Existe pelo menos uma diferença na média das horas em atividade entre os grupos de sintomas de depressão')
else:
    print('H0 aceita: Não existe diferença entre os grupos de depressão com relação à média das horas em atividade')

In [ ]:
tukey = pairwise_tukeyhsd(df_teste['PAG_HRW_log'], df_teste['DEPRESINTGROUP'])
print(tukey)

- H0: Não existe diferença entre os grupos de depressão com relação ao escore total
- H1: Existe pelo menos uma diferença na média ao escore total entre os grupos de sintomas de depressão

In [ ]:
grafico_boxplot_grp(df_merged_2.replace(label_quali), "HEI2015_TOTAL_SCORE", "HEI - Escore total")

In [ ]:
df_teste = df_merged_2[["DEPRESINTGROUP", "HEI2015_TOTAL_SCORE"]].dropna()

stat, p = f_oneway(df_teste[(df_teste.DEPRESINTGROUP == 0)]["HEI2015_TOTAL_SCORE"],
                   df_teste[(df_teste.DEPRESINTGROUP == 1)]["HEI2015_TOTAL_SCORE"],
                   df_teste[(df_teste.DEPRESINTGROUP == 2)]["HEI2015_TOTAL_SCORE"])

if p < 0.05:
    print('H0 rejeitada: Existe pelo menos uma diferença na média ao escore total entre os grupos de sintomas de depressão')
else:
    print('H0 aceita: Não existe diferença entre os grupos de depressão com relação ao escore total')

In [ ]:
tukey = pairwise_tukeyhsd(df_teste['HEI2015_TOTAL_SCORE'], df_teste['DEPRESINTGROUP'])
print(tukey)

### Perfil de hábitos saudáveis

In [ ]:
# Alimentação saudável e Exercícios Físicos

sns.boxplot(y = "ADHERENCE", x = "HEI2015_TOTAL_SCORE", orient = "h", data = df_merged_2.replace(label_quali));

In [ ]:
fig, ax = plt.subplots(ncols = 2, figsize = (15,5))

sns.regplot(x = 'HEI2015_TOTAL_SCORE', y = 'PAG_HRW', lowess = True, line_kws = {'color': 'red'}, data = df_merged_2, ax = ax[0])
sns.regplot(x = 'HEI2015_TOTAL_SCORE', y = 'PAG_HRW_log', lowess=True, line_kws={'color': 'red'}, data = df_merged_2, ax = ax[1]);

- H0: Não existe diferença entre a aderência com relação ao escore total
- H1: Existe pelo menos uma diferença na média ao escore total entre a aderência

In [ ]:
df_teste = df_merged_2[["ADHERENCE", "HEI2015_TOTAL_SCORE"]].dropna()

stat, p = f_oneway(df_teste[(df_teste.ADHERENCE == 1)]["HEI2015_TOTAL_SCORE"],
                   df_teste[(df_teste.ADHERENCE == 2)]["HEI2015_TOTAL_SCORE"],
                   df_teste[(df_teste.ADHERENCE == 3)]["HEI2015_TOTAL_SCORE"])

if p < 0.05:
    print('H0 rejeitada: Existe pelo menos uma diferença na média do escore total entre a aderência')
else:
    print('H0 aceita: Não existe diferença entre a aderência com relação ao escore total')

In [ ]:
tukey = pairwise_tukeyhsd(df_teste['HEI2015_TOTAL_SCORE'], df_teste['ADHERENCE'])
print(tukey)

In [ ]:
# Hábitos saudáveis e Gênero

plt.figure(figsize=(7,5))
sns.boxplot(x = "RIAGENDR", y = "HEI2015_TOTAL_SCORE", data = df_merged_2.replace(label_quali));

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(15,5))

sns.boxplot(x = "RIAGENDR", y = "PAG_HRW", data = df_merged_2.replace(label_quali), ax = ax[0])
sns.boxplot(x = "RIAGENDR", y = "PAG_HRW_log", data = df_merged_2.replace(label_quali), ax = ax[1]);

- H0: Não existe pelo menos uma diferença na média do escore total entre os gêneros
- H1: Existe pelo menos uma diferença na média do escore total entre os gêneros

In [ ]:
df_teste = df_merged_2[["RIAGENDR", "PAG_HRW", "PAG_HRW_log", "HEI2015_TOTAL_SCORE"]].dropna()

stat, p = f_oneway(df_teste[(df_teste.RIAGENDR == 1)]["HEI2015_TOTAL_SCORE"],
                   df_teste[(df_teste.RIAGENDR == 2)]["HEI2015_TOTAL_SCORE"])

if p < 0.05:
    print('H0 rejeitada: Existe pelo menos uma diferença na média do escore total entre os gêneros')
else:
    print('H0 aceita: Não existe diferença entre os gêneros com relação ao escore total')

In [ ]:
tukey = pairwise_tukeyhsd(df_teste['HEI2015_TOTAL_SCORE'], df_teste['RIAGENDR'])
print(tukey)

- H0: Não existe pelo menos uma diferença na média das horas em atividade entre os gêneros
- H1: Existe pelo menos uma diferença na média das horas em atividade entre os gêneros

In [ ]:
stat, p = f_oneway(df_teste[(df_teste.RIAGENDR == 1)]["PAG_HRW_log"],
                   df_teste[(df_teste.RIAGENDR == 2)]["PAG_HRW_log"])

if p < 0.05:
    print('H0 rejeitada: Existe pelo menos uma diferença na média de horas em atividades entre os gêneros')
else:
    print('H0 aceita: Não existe diferença entre os gêneros com relação a média de horas em atividade')

In [ ]:
tukey = pairwise_tukeyhsd(df_teste['PAG_HRW_log'], df_teste['RIAGENDR'])
print(tukey)

In [ ]:
# Hábitos saudáveis e Raça/etnia

plt.figure(figsize=(7,5))
sns.boxplot(x = "RIDRETH1", y = "HEI2015_TOTAL_SCORE", data = df_merged_2.replace(label_quali));

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(15,5))

sns.boxplot(x = "RIDRETH1", y = "PAG_HRW", data = df_merged_2.replace(label_quali), ax = ax[0])
sns.boxplot(x = "RIDRETH1", y = "PAG_HRW_log", data = df_merged_2.replace(label_quali), ax = ax[1]);

- H0: Não existe pelo menos uma diferença na média do escore total e a raça/etnia
- H1: Existe pelo menos uma diferença na média do escore total e a raça/etnia

In [ ]:
df_teste = df_merged_2[["RIDRETH1", "PAG_HRW", "PAG_HRW_log", "HEI2015_TOTAL_SCORE"]].dropna()

stat, p = f_oneway(df_teste[(df_teste.RIDRETH1 == 1)]["HEI2015_TOTAL_SCORE"],
                   df_teste[(df_teste.RIDRETH1 == 2)]["HEI2015_TOTAL_SCORE"],
                   df_teste[(df_teste.RIDRETH1 == 3)]["HEI2015_TOTAL_SCORE"],
                   df_teste[(df_teste.RIDRETH1 == 4)]["HEI2015_TOTAL_SCORE"])

if p < 0.05:
    print('H0 rejeitada: Existe pelo menos uma diferença na média do escore total e a raça/etnia')
else:
    print('H0 aceita: Não existe pelo menos uma diferença na média do escore total e a raça/etnia')

In [ ]:
tukey = pairwise_tukeyhsd(df_teste['HEI2015_TOTAL_SCORE'], df_teste['RIDRETH1'])
print(tukey)

- H0: Não existe pelo menos uma diferença na média de atividade em horas e a raça/etnia
- H1: Existe pelo menos uma diferença na média de atividade em horas e a raça/etnia

In [ ]:
stat, p = f_oneway(df_teste[(df_teste.RIDRETH1 == 1)]["PAG_HRW_log"],
                   df_teste[(df_teste.RIDRETH1 == 2)]["PAG_HRW_log"],
                   df_teste[(df_teste.RIDRETH1 == 3)]["PAG_HRW_log"],
                   df_teste[(df_teste.RIDRETH1 == 4)]["PAG_HRW_log"])

if p < 0.05:
    print('H0 rejeitada: Existe pelo menos uma diferença na média de atividade em horas e a raça/etnia')
else:
    print('H0 aceita: Não existe pelo menos uma diferença na média de atividade em horas e a raça/etnia')

In [ ]:
tukey = pairwise_tukeyhsd(df_teste['PAG_HRW_log'], df_teste['RIDRETH1'])
print(tukey)

## 6) Aspectos da análise

1) O tipo de estudo empregado pelo NHANES é observacional visto que não houve intervenção dos pesquisadores.

2) Quanto as possíveis fontes de viés, está a de muitas pessoas que não responderam ou que não sabiam responder.

3) A partir da análise realizada, é possível visualizar que hábitos saudáveis possuem uma correlação em menores índices depressivos. Porém, correlação é diferente de causalidade e para tal, deve ser feito um estudo mais profundo.

4) A análise falha em conter uma parte sobre histórico familiar. Além disso, os índices podem enganar pois nem sempre os sintomas de depressão são depressão de fato, podendo ser, por exemplo, cansaço, que pode estar relacionado à jornadas demasiadamente longas de trabalho.

5) Informações sobre o histórico familiar são bem vindas, também há quanto tempo prática atividade física, doenças crônicas que poderiam estar associadas a algum dos quadros (impactando ou na frequência de atividade física ou nos sintomas de depressão) e alguma informação demográfica relacionada à localidade do entrevistado (como, por exemplo, se moradores da zona urbana possuem mais sintomas depressivos que os da zona rural, entre outros).